**nota**: In questo Colab viene valutato se è sensato reintrodurre le 4176 righe che presentano valori nulli **solo** nelle colonne delle lunghezze: non ha senso perché sia le categories1 che le categories2 hanno tutte somma pari a 0 (tutte i 4176 utenti hanno tutte le entrate di categories1 e di categories2 pari a 0). 

# Importazione dati

In [59]:
from google.colab import drive 
import pandas as pd
import re
from glob import glob
import numpy as np
from math import exp

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay

from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC

import scipy

In [60]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Perché il seguente comando funzioni è necessario aggiungere una scorciatoia a DS Lab - PROGETTO al proprio Drive.

In [61]:
#!cp -av '/content/gdrive/My Drive/DS Lab - PROGETTO/Dataset aggregati' 'campaignClickDataset'

Lista dei file contenenti i dati

In [62]:
#part_files = sorted(glob('campaignClickDataset/part*.csv'))

In [63]:
#df = pd.concat((pd.read_csv(file) for file in part_files), ignore_index=True)

In [64]:
# Export in csv
#pd.DataFrame.to_csv(df.set_index('ad_form_id'), '/content/gdrive/My Drive/DS Lab - PROGETTO/Ottobre/dataset_completo.csv')

Importazione dei dati, solo csv completo

In [98]:
df = pd.read_csv('/content/gdrive/My Drive/DS Lab - PROGETTO/Ottobre/dataset_completo.csv')

In [66]:
df.head()

,ad_form_id,suspicious,clicks,impressions,buy,os_android,os_bsd,os_ios,os_linux,os_osx,...,feelings1_surprise,feelings1_sympathy,feelings1_thirst,feelings1_thoughtful,feelings1_torment,feelings1_traditionalist,feelings1_trust,feelings1_uncategorized,feelings1_violence,feelings1_wealth
0,7241135337730459930,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8408205703932483258,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5884145627470739741,0,0,2,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2463214613830580928,0,0,6,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6157419832685345406,0,0,3,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Controllo valori nulli

Controllo in quali colonne si trovano i `NaN`.

In [67]:
col_nan = []
for col in df.columns:
  if len(df[df[col].isnull()]) != 0:
    print(col, len(df[df[col].isnull()]))
    col_nan.append(col)

L00_50 4178
L51_100 20033
L101_250 20028
L251_500 20026
L501_1000 20033
L1001_2500 20032
L2501_5000 20033
L5001_10000 20033
L10001_more 20033


I `NaN` si trovano solo nelle colonne lunghezza e non tutte le 20033 righe eliminate dopo presentano `NaN`. Vado a vedere cosa succede in queste righe.

In [68]:
df[df['L10001_more'].isnull()][col_nan]

,L00_50,L51_100,L101_250,L251_500,L501_1000,L1001_2500,L2501_5000,L5001_10000,L10001_more
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
82529,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82534,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82535,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ci sono gli `inf`, che di fatto possiamo trattare come `NaN`.

Vado allora a guardare il numero di `NaN` e di inf in ogni colonna

In [69]:
col_infami = []
for col in df.columns:
  if (len(df[df[col].isnull()]) != 0) or (len(df[df[col] == np.inf]) != 0):
    diz = {}
    diz['Colonna'] = col
    diz['Numero di NaN'] = len(df[df[col].isnull()])
    diz['Numero di inf'] = len(df[df[col] == np.inf])
    diz['Numero totale di NaN e inf'] = len(df[df[col].isnull()]) + len(df[df[col] == np.inf])

    col_infami.append(diz)

In [70]:
# Numero di colonne che contengono almeno un NaN o almeno un inf
len(col_infami)

50

In [71]:
col_infami

[{'Colonna': 'L00_50',
  'Numero di NaN': 4178,
  'Numero di inf': 15855,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L51_100',
  'Numero di NaN': 20033,
  'Numero di inf': 0,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L101_250',
  'Numero di NaN': 20028,
  'Numero di inf': 5,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L251_500',
  'Numero di NaN': 20026,
  'Numero di inf': 7,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L501_1000',
  'Numero di NaN': 20033,
  'Numero di inf': 0,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L1001_2500',
  'Numero di NaN': 20032,
  'Numero di inf': 1,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L2501_5000',
  'Numero di NaN': 20033,
  'Numero di inf': 0,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L5001_10000',
  'Numero di NaN': 20033,
  'Numero di inf': 0,
  'Numero totale di NaN e inf': 20033},
 {'Colonna': 'L10001_more',
  'Numero di NaN': 20033,
  'Numero di inf': 0,
  'Numero to

Ci sono 50 colonne che contengono almeno un `NaN` o almeno un `inf`. È curioso che per le colonne lunghezza la somma di `Nan` e `inf` è sempre 20033.

Ora vado a guardare se per ognuno dei 20033 utenti che eliminiamo, ci sono `NaN` o `inf` solo nelle colonne lunghezza o se ci sono o `inf` anche in altre colonne (sicuramente infatti non ci sono `NaN` perché abbiamo verificato che si trovano solo nelle colonne lunghezza.

In [72]:
# Costruisco la lista di colonne che contengono i NaN e gli inf
col_nan_inf = []
for diz in col_infami:
  col_nan_inf.append(diz['Colonna'])

col_nan_inf

['L00_50',
 'L51_100',
 'L101_250',
 'L251_500',
 'L501_1000',
 'L1001_2500',
 'L2501_5000',
 'L5001_10000',
 'L10001_more',
 'categories1_business',
 'categories1_finance',
 'categories1_society',
 'categories1_technologyandcomputing',
 'categories1_uncategorized',
 'categories2_advertising',
 'categories2_business',
 'categories2_company',
 'categories2_crime',
 'categories2_finance',
 'categories2_internettechnology',
 'categories2_marketing',
 'categories2_software',
 'categories2_technologyandcomputing',
 'categories2_uncategorized',
 'categories3_advertising',
 'categories3_business',
 'categories3_company',
 'categories3_finance',
 'categories3_internettechnology',
 'categories3_marketing',
 'categories3_propertycrime',
 'categories3_software',
 'categories3_technologyandcomputing',
 'categories3_uncategorized',
 'admants1_businessandindustrialsupplies',
 'admants1_finance',
 'admants1_lawfirms',
 'admants1_news',
 'admants1_onlinepayments',
 'admants1_software',
 'admants1_stre

In [73]:
# Rimuovo dalla lista le colonne lunghezza (le prime 8 entrate di col_nan_inf)
for _ in range(9):
  col_nan_inf.pop(0)

# Aggiungo ad_form_id per facilitare l'identificazione degli utenti
col_nan_inf.append('ad_form_id')

col_nan_inf

['categories1_business',
 'categories1_finance',
 'categories1_society',
 'categories1_technologyandcomputing',
 'categories1_uncategorized',
 'categories2_advertising',
 'categories2_business',
 'categories2_company',
 'categories2_crime',
 'categories2_finance',
 'categories2_internettechnology',
 'categories2_marketing',
 'categories2_software',
 'categories2_technologyandcomputing',
 'categories2_uncategorized',
 'categories3_advertising',
 'categories3_business',
 'categories3_company',
 'categories3_finance',
 'categories3_internettechnology',
 'categories3_marketing',
 'categories3_propertycrime',
 'categories3_software',
 'categories3_technologyandcomputing',
 'categories3_uncategorized',
 'admants1_businessandindustrialsupplies',
 'admants1_finance',
 'admants1_lawfirms',
 'admants1_news',
 'admants1_onlinepayments',
 'admants1_software',
 'admants1_streamingservices',
 'admants1_telecommunications',
 'feelings1_behaviour',
 'feelings1_emotions',
 'feelings1_frugal',
 'feeling

In [74]:
# Salvo in una variabile gli id che eliminiamo
id_nan = list(df[df['L10001_more'].isnull()]['ad_form_id'])

In [75]:
# Costruisco ora un dataset con solo le colonne di col_nan_inf e solo gli id che eliminiamo (perché non ci sono
# altri NaN o inf in altre righe >> questo è verificato nella normalizzazione delle lunghezze)
df2 = pd.DataFrame()
for id in id_nan:
  df2 = df2.append(df[df['ad_form_id'] == id][col_nan_inf])
df2

,categories1_business,categories1_finance,categories1_society,categories1_technologyandcomputing,categories1_uncategorized,categories2_advertising,categories2_business,categories2_company,categories2_crime,categories2_finance,...,admants1_telecommunications,feelings1_behaviour,feelings1_emotions,feelings1_frugal,feelings1_love,feelings1_modern,feelings1_sedentary,feelings1_success,feelings1_traditionalist,ad_form_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7241135337730459930
2,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5884145627470739741
3,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2463214613830580928
6,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4479359120959384569
11,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8996170844411030610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82529,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341600422323320625
82532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3756819457074416708
82534,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1193695853517800236
82535,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6175724090840247579


In [76]:
# Sostituisco gli inf con dei NaN sapendo che non ci sono NaN in df2 -> questa operazione serve per la cella 
# successiva
df2.replace(np.inf, np.nan, inplace=True)

In [77]:
# Ora vado a guardare il numero di NaN in ogni riga e lo salvo in una lista
number_of_nan = df2.isnull().sum(axis=1).tolist()

In [78]:
# Ora verifico se ci sono entrate di number_of_nan pari a 0 => se ce ne sono, vuol dire che ci sono utenti per cui
# i valori nulli si trovano solo nelle lunghezze
count = 0
for non in number_of_nan:
  if non == 0:
    count += 1

count

4176

**Conclusione:** ci sono 4176 utenti su 20033 che hanno un `NaN` o un `inf` solo nelle colonne lunghezza. Tutti gli altri hanno almeno un `inf` in una delle altre 41 colonne che presentano valori nulli.

In [79]:
# Aggiungo una colonna con il numero di valori nulli in ogni riga
df2['Somma di NaN e inf'] = number_of_nan
df2

,categories1_business,categories1_finance,categories1_society,categories1_technologyandcomputing,categories1_uncategorized,categories2_advertising,categories2_business,categories2_company,categories2_crime,categories2_finance,...,feelings1_behaviour,feelings1_emotions,feelings1_frugal,feelings1_love,feelings1_modern,feelings1_sedentary,feelings1_success,feelings1_traditionalist,ad_form_id,Somma di NaN e inf
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7241135337730459930,0
2,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5884145627470739741,3
3,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2463214613830580928,3
6,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4479359120959384569,3
11,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8996170844411030610,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82529,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,341600422323320625,3
82532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3756819457074416708,0
82534,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1193695853517800236,3
82535,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6175724090840247579,3


In [80]:
# Salvo i 4176 id in una lista che useremo per costruire un sottoinsieme di df contenente solo le 4176 righe notevoli che reintrodurremo
id_4176 = list(df2[df2['Somma di NaN e inf'] == 0]['ad_form_id'])

In [99]:
# Costruisco un dataframe con solo i 4176 utenti che ci interessano
df3 = pd.DataFrame()
for id in id_4176:
  try:
    df3 = df3.append(df[df['ad_form_id'] == id])
  except:
    pass

In [100]:
df3

,ad_form_id,suspicious,clicks,impressions,buy,os_android,os_bsd,os_ios,os_linux,os_osx,...,feelings1_surprise,feelings1_sympathy,feelings1_thirst,feelings1_thoughtful,feelings1_torment,feelings1_traditionalist,feelings1_trust,feelings1_uncategorized,feelings1_violence,feelings1_wealth
0,7241135337730459930,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,1633170663585866342,0,0,6,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,8800313386888394827,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,7609729276934938609,0,0,2,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,2143737829494737521,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82472,4756679042780103789,0,0,4,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82478,8206347819790609161,0,0,3,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82494,989818539064099103,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82495,5299665908543523273,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Colonne
Idee per la rimozione delle colonne:
* soli 0
  * buy
  * sentiments
* identiche informazioni
  * time2, stessa informazione di time1
* correlazione + identiche informazioni
  * 57% degli admants correlate con categories1 -> dopo aver eliminato le categories che causano i nan nel calcolo della correlazione i risultati non sono cambiati
* uguaglianza delle distribuzioni + identiche informazioni
  * categories 1, 2 e 3 => rappresentano le stesse informazioni, ma le c1 sono più diverse a livello di distribuzione tra click e non click rispetto a c2 e c3
  * feelings => la quasi totalità delle colonne (93%) ha la stessa distribuzione tra click e non click (sono di fatto quasi tutti 0) -> la percentuale di 0 nelle colonne è minore del 99% solo in 4 casi: due colonne hanno % di 0 pari a circa il 98%, mentre le altre 2 hanno % di 0 minore del 77%. Queste ultime due colonne hanno anche distribuzione diversa tra click e non click e % di 0 diverse tra click e non click. Una possibile idea è quella di mantenere 3 colonne, queste due più una terza, feelings_others, in cui accorpiamo tutti gli altri feelings. Tuttavia, più di 44000 righe hanno tutti i feelings pari a 0 e questo sembra una mancanza di informazione importante e quindi si può anche valutare di eliminare tutte le colonne feelings.

In [101]:
# Eliminazione con impressions = 0 (click passati da email) e suspicious = 1 (sospette frodi)

df  = df[df['impressions'] != 0]
df = df[df['suspicious'] == 0]

In [102]:
# eliminazione colonna sus

df.drop(columns='suspicious', inplace=True)

In [103]:
colonne = list(df.columns)
for colonna in colonne:
  print(colonna)

ad_form_id
clicks
impressions
buy
os_android
os_bsd
os_ios
os_linux
os_osx
os_other
os_windows
device_type
browser_android
browser_chrome
browser_chromium
browser_edge
browser_firefox
browser_ie
browser_opera
browser_other
browser_safari
browser_unknown
time1_workday_morning
time1_workday_afternoon
time1_workday_evening
time1_workday_night
time1_weekend_morning
time1_weekend_afternoon
time1_weekend_evening
time1_weekend_night
time2_morning_early
time2_morning
time2_launch
time2_afternoon
time2_evening
time2_night
time2_sleep
L00_50
L51_100
L101_250
L251_500
L501_1000
L1001_2500
L2501_5000
L5001_10000
L10001_more
categories1_artandentertainment
categories1_automotive
categories1_business
categories1_careers
categories1_education
categories1_emotions
categories1_familyandparenting
categories1_finance
categories1_foodanddrink
categories1_healthandfitness
categories1_hobbiesandinterests
categories1_homeandgarden
categories1_intentions
categories1_lawgovtandpolitics
categories1_news
categor

## Buy

Togliamo buy perché è totalmente popolata da 0 e eventuali valori pari a 1 indicherebbero che una persona ha cliccato dopo aver ricevuto la pubblicità => è un'informazione che non abbiamo prima di aver mandato la pubblicità.

In [104]:
df.drop('buy', axis=1, inplace=True)

## Variabili Lunghezza
Per ora teniamo le colonne come sono, senza aggregarle. Poi eventualmente le aggreghiamo sulla base dei risultati di SVM.

### Normalizzazione

Normalizziamo le colonne (esattamente come abbiamo fatto a luglio).

Dall'analisi risulta che si sono circa 20000 righe con valori mancanti (nan o infiniti) per almeno una delle colonne. L'eliminazione la facciamo sulla base della somma (eliminiamo se è nan o infinito).

In [105]:
L = re.compile('^L+.*', re.IGNORECASE)
lung = []

for v in df.columns:
  if L.search(v):
    print(v)
    lung.append(v)

L00_50
L51_100
L101_250
L251_500
L501_1000
L1001_2500
L2501_5000
L5001_10000
L10001_more


In [106]:
# Esempio di ciò che succede coi NaN
df[df['L101_250'].isnull()][lung]

,L00_50,L51_100,L101_250,L251_500,L501_1000,L1001_2500,L2501_5000,L5001_10000,L10001_more
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
82529,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82534,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82535,inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# Somma dei valori nelle colonne
def somma_lung(riga):
  sum = 0
  for el in lung:
    sum += riga[el]
  return sum

df['somma'] = df.apply(somma_lung, axis=1)
df[['somma']]

,somma
0,NaN
1,237.5
2,NaN
3,NaN
4,100.0
...,...
82559,100.0
82560,100.0
82561,100.0
82562,100.0


In [108]:
# Verifichiamo se ci sono valori infiniti
from numpy import inf

sum(df['somma'] == inf) 

0

In [109]:
# Verifichiamo se ci sono valori nulli
sum(df['somma'].isnull())

20011

Eliminiamo ora tutte queste righe.

In [110]:
indexNames = df[(df['somma'].isnull())].index
df.drop(indexNames, inplace=True)
df['somma'].shape

(62379,)

Ora normalizziamo

In [111]:
for l in lung:
  df[l] = round((df[l] / df['somma']) * 100, 2)

df['somma'] = df.apply(somma_lung, axis=1)

In [112]:
# controllo
df[['L00_50', 'L51_100', 'L101_250', 'L251_500', 'L501_1000', 'L1001_2500', 'L2501_5000', 'L5001_10000', 'L10001_more', 'somma']]

,L00_50,L51_100,L101_250,L251_500,L501_1000,L1001_2500,L2501_5000,L5001_10000,L10001_more,somma
1,100.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4,100.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
5,0.00,0.00,0.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0
7,66.67,33.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
8,42.86,57.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...,...
82559,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
82560,100.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
82561,100.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
82562,11.11,88.89,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0


In [113]:
# Delete coloumn 'somma'
df.drop(columns=['somma'], inplace=True)

In [114]:
# Controllo se ci sono colonne con valori pari a inf
for col in df.columns:
  if len(df[df[col] == np.inf]) != 0:
    print(col)

## Reintroduzione delle 4176 righe che hanno NaN solo nelle lunghezze e sostituzione dei NaN

In [115]:
df3

,ad_form_id,suspicious,clicks,impressions,buy,os_android,os_bsd,os_ios,os_linux,os_osx,...,feelings1_surprise,feelings1_sympathy,feelings1_thirst,feelings1_thoughtful,feelings1_torment,feelings1_traditionalist,feelings1_trust,feelings1_uncategorized,feelings1_violence,feelings1_wealth
0,7241135337730459930,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,1633170663585866342,0,0,6,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,8800313386888394827,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,7609729276934938609,0,0,2,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,2143737829494737521,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82472,4756679042780103789,0,0,4,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82478,8206347819790609161,0,0,3,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82494,989818539064099103,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82495,5299665908543523273,0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
print(f"{len(df)} + {len(df3)} = {len(df)+len(df3)}") # dimensione attesa

62379 + 4176 = 66555


In [120]:
df = df.append(df3)

In [121]:
df

,ad_form_id,clicks,impressions,os_android,os_bsd,os_ios,os_linux,os_osx,os_other,os_windows,...,feelings1_thirst,feelings1_thoughtful,feelings1_torment,feelings1_traditionalist,feelings1_trust,feelings1_uncategorized,feelings1_violence,feelings1_wealth,suspicious,buy
1,8408205703932483258,0,1,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,6157419832685345406,0,3,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
5,6336227050936118437,0,3,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
7,6272249560265519045,0,5,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
8,2248420359615433360,0,5,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82472,4756679042780103789,0,4,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82478,8206347819790609161,0,3,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82494,989818539064099103,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82495,5299665908543523273,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Variabili os, browser e device

Tipi:
* os e browser sono composte da più variabili binarie;
* device è categorica, quindi si occuperà `OneHotEncoder` di binarizzarla.

Possiamo accorpare sulla base della loro distribuzione.

### os Distribuzione

In [122]:
os = ('os_android', 'os_bsd', 'os_ios', 'os_linux', 'os_osx', 'os_other', 'os_windows')

In [123]:
for tipo in os:
  yes = df[df['clicks'] > 0][tipo]
  no = df[df['clicks'] <= 0][tipo]

  zinyes = 0
  for r in yes:
    if r <= 0:
      zinyes = zinyes +1

  zinno = 0
  for r in no:
    if r <= 0:
      zinno = zinno +1

  print(tipo, ':')
  print('clicker che usano questo tipo: ', len(yes)-zinyes, 'su ', len(yes), ' -> ', 1-zinyes/len(yes))
  print('non clicker che usano questo tipo: ', len(no)-zinno, 'su ', len(no), ' -> ', 1-zinno/len(no))
  print('')

os_android :
clicker che usano questo tipo:  17 su  199  ->  0.085427135678392
non clicker che usano questo tipo:  5814 su  66356  ->  0.0876183012839834

os_bsd :
clicker che usano questo tipo:  0 su  199  ->  0.0
non clicker che usano questo tipo:  1 su  66356  ->  1.5070227258995317e-05

os_ios :
clicker che usano questo tipo:  0 su  199  ->  0.0
non clicker che usano questo tipo:  1 su  66356  ->  1.5070227258995317e-05

os_linux :
clicker che usano questo tipo:  2 su  199  ->  0.01005025125628145
non clicker che usano questo tipo:  466 su  66356  ->  0.007022725902706584

os_osx :
clicker che usano questo tipo:  24 su  199  ->  0.12060301507537685
non clicker che usano questo tipo:  9592 su  66356  ->  0.1445536198685876

os_other :
clicker che usano questo tipo:  0 su  199  ->  0.0
non clicker che usano questo tipo:  7 su  66356  ->  0.00010549159081318926

os_windows :
clicker che usano questo tipo:  156 su  199  ->  0.7839195979899498
non clicker che usano questo tipo:  50475 s

### browser Distribuzione

In [124]:
brow = ('browser_android', 'browser_chrome', 'browser_chromium', 'browser_edge', 'browser_firefox', 'browser_ie', 'browser_opera', 
        'browser_other', 'browser_safari', 'browser_unknown')

In [125]:
for tipo in brow:
  yes = df[df['clicks'] > 0][tipo]
  no = df[df['clicks'] <= 0][tipo]

  zinyes = 0
  for r in yes:
    if r <= 0:
      zinyes = zinyes +1

  zinno = 0
  for r in no:
    if r <= 0:
      zinno = zinno +1

  print(tipo, ':')
  print('clicker che usano questo tipo: ', len(yes)-zinyes, 'su ', len(yes), ' -> ', 1-zinyes/len(yes))
  print('non clicker che usano questo tipo: ', len(no)-zinno, 'su ', len(no), ' -> ', 1-zinno/len(no))
  print('')

browser_android :
clicker che usano questo tipo:  1 su  199  ->  0.005025125628140725
non clicker che usano questo tipo:  172 su  66356  ->  0.0025920790885526346

browser_chrome :
clicker che usano questo tipo:  116 su  199  ->  0.5829145728643217
non clicker che usano questo tipo:  44161 su  66356  ->  0.6655163059858943

browser_chromium :
clicker che usano questo tipo:  0 su  199  ->  0.0
non clicker che usano questo tipo:  0 su  66356  ->  0.0

browser_edge :
clicker che usano questo tipo:  48 su  199  ->  0.24120603015075381
non clicker che usano questo tipo:  10050 su  66356  ->  0.151455783953222

browser_firefox :
clicker che usano questo tipo:  15 su  199  ->  0.07537688442211055
non clicker che usano questo tipo:  3578 su  66356  ->  0.05392127313279882

browser_ie :
clicker che usano questo tipo:  3 su  199  ->  0.015075376884422065
non clicker che usano questo tipo:  2753 su  66356  ->  0.04148833564410148

browser_opera :
clicker che usano questo tipo:  0 su  199  ->  0.0

### device Distribuzione

In [126]:
len(df[df['clicks']>0])

199

In [127]:
for n in (1,2,3):
  users = len(df[df['device_type']==n])
  c_users = len( df[(df['device_type']==n) & (df['clicks']>0)] )
  nc_users = len( df[(df['device_type']==n) & (df['clicks']<=0)] )
  if(n==1): print('Mobile')
  if(n==2): print('Desktop and Laptop')
  if(n==3): print('Unknown/Others')
  print('clicker che usano questo device: ', c_users, 'su ', 210, ' -> ', c_users/210)
  print('non clicker che usano questo device: ', nc_users, 'su ', len(df)-210, ' -> ', nc_users/(len(df)-210))
  print('utilizzatori totali: ', users)
  print(' ')

Mobile
clicker che usano questo device:  17 su  210  ->  0.08095238095238096
non clicker che usano questo device:  5820 su  66345  ->  0.08772326475243047
utilizzatori totali:  5837
 
Desktop and Laptop
clicker che usano questo device:  182 su  210  ->  0.8666666666666667
non clicker che usano questo device:  60533 su  66345  ->  0.9123973170547893
utilizzatori totali:  60715
 
Unknown/Others
clicker che usano questo device:  0 su  210  ->  0.0
non clicker che usano questo device:  3 su  66345  ->  4.521817770743839e-05
utilizzatori totali:  3
 


## Variabili Time

Contenendo time1 e time2 informazioni simili (momento della giornata in cui la pagina è stata visitata), ma avendo time2 infromazioni mancanti per circa 3000 righe, si è deciso di mantenere le sole colonne di time1.

Viene quindi
* mostrato che le time2 hanno informazioni mancanti (somme a zero);
* mostrare che le time1 hanno tutte le informazioni.

Passi successivi:
* rimuovere effettivamente le Time2;
* normalizzare le Time1.


### Controlli

In [128]:
# colonne da utilizzare
colonne = df.columns.to_numpy()
t = re.compile('^time+.*', re.IGNORECASE)
t1 = re.compile('^time1+.*', re.IGNORECASE)
times1 = []
times2 = []

for v in colonne:
  if t.search(v):
    print(v)
    if t1.search(v):
      times1.append(v)
    else:
      times2.append(v)

time1_workday_morning
time1_workday_afternoon
time1_workday_evening
time1_workday_night
time1_weekend_morning
time1_weekend_afternoon
time1_weekend_evening
time1_weekend_night
time2_morning_early
time2_morning
time2_launch
time2_afternoon
time2_evening
time2_night
time2_sleep


In [129]:
# calcolo la somma dei tempi1 e tempi2

df['sommat1'] = (df['time1_workday_morning'] + df['time1_workday_afternoon'] + df['time1_workday_evening'] + df['time1_workday_night'] +
                 df['time1_weekend_morning'] + df['time1_weekend_afternoon'] + df['time1_weekend_evening'] + df['time1_weekend_night'])

df['sommat2'] = (df['time2_morning_early'] + df['time2_morning'] + df['time2_launch'] + df['time2_afternoon'] + df['time2_evening'] +
                 df['time2_night'] + df['time2_sleep'])

In [130]:
# non sono presenti somme infinite
len(df[(df['sommat1'] > 100000) | (df['sommat2'] > 100000)][['sommat1', 'sommat2']])

0

In [131]:
# sono presenti più di 3000 righe che mostrano somma per tempi2 pari a zero (13 hanno cliccato)
print(len(df[df['sommat2'] == 0][['sommat1', 'sommat2']]))
print(len(df[(df['sommat2'] == 0) & (df['clicks'] > 0)][['sommat1', 'sommat2']]))

3830
11


In [132]:
#  non sono presenti righe con somma a zero per tempi1
df[(df['sommat1'] == 0)][['sommat1', 'sommat2']]

,sommat1,sommat2


In [133]:
# non sono presenti valori nulli
print(len(df[df['sommat1'].isnull()]))
print(len(df[df['sommat2'].isnull()]))

0
0


### Normalizzazione time1, eliminazione time2

In [134]:
# Riscalo i tempi(1) in modo che somma sia 100 
for t2 in times1:
  df[t2] = round(df[t2]/ df['sommat1']*100, 2)

times1.append('sommat1')
df[times1].head(5)

,time1_workday_morning,time1_workday_afternoon,time1_workday_evening,time1_workday_night,time1_weekend_morning,time1_weekend_afternoon,time1_weekend_evening,time1_weekend_night,sommat1
1,26.32,73.68,0.00,0.0,0.0,0.0,0.0,0.0,237.5
4,0.00,0.00,0.00,100.0,0.0,0.0,0.0,0.0,100.0
5,0.00,100.00,0.00,0.0,0.0,0.0,0.0,0.0,100.0
7,0.00,100.00,0.00,0.0,0.0,0.0,0.0,0.0,100.0
8,14.29,14.29,71.43,0.0,0.0,0.0,0.0,0.0,175.0


In [135]:
# elimino colonne di time2 e delle somme
df.drop(columns=['sommat1', 'sommat2'], inplace=True)
df.drop(columns=times2, inplace=True)

## Variabili Categories

Normalizziamo le colonne (come abbiamo fatto a luglio, includendo però anche le categories 2 e 3).

### Normalizzazione

#### Categories1

In [138]:
# Colonne categories1
cat1 = re.compile('categories1_+.*', re.IGNORECASE)
cats1 = []
for label in colonne:
  occ = cat1.findall(label)
  if occ:
    cats1.append(occ[0])
  
# Calcolo somma
def somma_cat1(riga):
  sum = 0
  for cat in cats1:
    sum += riga[cat]
  return(round(sum, 1))

df['somma1'] = df.apply(somma_cat1, axis=1)

In [139]:
# controllo quanti nan ci sono nelle somme: non ci sono nan

df[df['somma1'].isnull()][['ad_form_id', 'somma1']]

,ad_form_id,somma1


In [140]:
# controllo se ci sono nan nelle categorie singole: non ci sono

for v in cats1:
  nn = len(df[df[v].isnull()])
  if (nn != 0):
    print(v, nn)

In [141]:
# controllo quanti inf ci sono nelle somme: non ci sono inf
df[df['somma1'] == np.inf][['ad_form_id', 'somma1']]

,ad_form_id,somma1


Le righe che davano somme infinite nelle lunghezze, sono le stesse che a luglio davano somme infinite nelle categories.

In [142]:
# controllo gli zero: non ce ne sono
df[df['somma1'] == 0][['ad_form_id', 'somma1']]

,ad_form_id,somma1
0,7241135337730459930,0.0
55,1633170663585866342,0.0
93,8800313386888394827,0.0
105,7609729276934938609,0.0
123,2143737829494737521,0.0
...,...,...
82472,4756679042780103789,0.0
82478,8206347819790609161,0.0
82494,989818539064099103,0.0
82495,5299665908543523273,0.0


In [143]:
# Normalizzo le somme a 100
for cat in cats1:
  df[cat] = round((df[cat] / df['somma1']) * 100, 2)

In [144]:
df[cats1]

,categories1_artandentertainment,categories1_automotive,categories1_business,categories1_careers,categories1_education,categories1_emotions,categories1_familyandparenting,categories1_finance,categories1_foodanddrink,categories1_healthandfitness,...,categories1_realestate,categories1_religionandspirituality,categories1_science,categories1_shopping,categories1_society,categories1_sports,categories1_styleandfashion,categories1_technologyandcomputing,categories1_travel,categories1_uncategorized
1,0.0,0.0,0.0,42.11,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,57.89
4,100.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00
5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.76,0.0,11.76,0.0,0.00
7,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,14.71,0.0,51.49,0.0,0.00
8,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,57.39,0.0,42.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df.drop('somma1', axis=1, inplace=True)

#### Categories 2

In [146]:
# Colonne categories2
cat2 = re.compile('categories2_+.*', re.IGNORECASE)
cats2 = []
for label in colonne:
  occ = cat2.findall(label)
  if occ:
    cats2.append(occ[0])
  
# Calcolo somma
def somma_cat2(riga):
  sum = 0
  for cat in cats2:
    sum += riga[cat]
  return(round(sum, 1))

df['somma2'] = df.apply(somma_cat2, axis=1)

In [147]:
# controllo quanti nan ci sono nelle somme: non ci sono nan

df[df['somma2'].isnull()][['ad_form_id', 'somma2']]

,ad_form_id,somma2


In [148]:
# controllo se ci sono nan nelle categorie singole: non ci sono

for v in cats2:
  nn = len(df[df[v].isnull()])
  if (nn != 0):
    print(v, nn)

In [149]:
# controllo quanti inf ci sono nelle somme: non ci sono inf
df[df['somma2'] == np.inf][['ad_form_id', 'somma2']]

,ad_form_id,somma2


Le righe che davano somme infinite nelle lunghezze, sono le stesse che a luglio davano somme infinite nelle categories.

In [150]:
# controllo gli zero: non ce ne sono
df[df['somma2'] == 0][['ad_form_id', 'somma2']]

,ad_form_id,somma2
0,7241135337730459930,0.0
55,1633170663585866342,0.0
93,8800313386888394827,0.0
105,7609729276934938609,0.0
123,2143737829494737521,0.0
...,...,...
82472,4756679042780103789,0.0
82478,8206347819790609161,0.0
82494,989818539064099103,0.0
82495,5299665908543523273,0.0


In [151]:
# Normalizzo le somme a 100
for cat in cats2:
  df[cat] = round((df[cat] / df['somma2']) * 100, 2)

In [152]:
df[cats2]

,categories2_accessories,categories2_addiction,categories2_adoption,categories2_adulteducation,categories2_adventuretravel,categories2_advertising,categories2_agriculture,categories2_alternativemedicine,categories2_alternativereligions,categories2_americancuisine,...,categories2_waterskiwakeboard,categories2_weather,categories2_weightlifting,categories2_weightloss,categories2_welfare,categories2_windsurfing,categories2_womenshealth,categories2_work,categories2_wrestling,categories2_zoo
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
df.drop('somma2', axis=1, inplace=True)